In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Substance_Use_Disorder/GSE161986'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Network preservation reveals shared and unique biological processes associated with chronic alcohol abuse in the NAc and PFC [mRNA]"
!Series_summary	"Chronic alcohol abuse has been linked to the disruption of executive function and allostatic conditioning of reward response dysregulation in the mesocorticolimbic pathway (MCL). Here, we analyzed genome-wide mRNA and miRNA expression from matched cases with alcohol dependence (AD) and controls (n=35) via gene network analysis to identify unique and shared biological processes dysregulated in the prefrontal cortex (PFC) and nucleus accumbens (NAc). We further investigated potential mRNA/miRNA interactions at the network and individual gene expression levels to identify the neurobiological mechanisms underlying AD in the brain. By using genotyped and imputed SNP data, we identified expression quantitative trait loci (eQTL) uncovering potential genetic regulatory elements for gene networks associated w

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if gene expression data is available.
# Based on the series summary, gene expression data (mRNA data) is present.
is_gene_available = True

# Data Availability Check
# Assign trait_row, age_row, and gender_row if the respective data is available.
# trait_row is assigned to the key 1 (diagnosis may refer to 'Substance_Use_Disorder')
# age_row is assigned to the key 2 as it directly refers to 'age'.
# gender_row is assigned to the key 3, noting that the gender is all male, so it is not variable.

trait_row = 1
age_row = 2
gender_row = 3

# Define the conversion functions
def convert_trait(value):
    # Extract value after the colon
    clean_value = value.split(':')[1].strip().lower()
    if clean_value == 'alcohol':  # Assuming 'alcohol' represents 'Substance_Use_Disorder'
        return 1
    elif clean_value == 'control':
        return 0
    return None

def convert_age(value):
    # Extract value after the colon
    try:
        return float(value.split(':')[1].strip())
    except ValueError:
        return None

def convert_gender(value):
    # Since all samples are male, directly return 1 for male
    # Extract value after the colon
    clean_value = value.split(':')[1].strip().lower()
    if clean_value == 'male':
        return 1
    elif clean_value == 'female':
        return 0
    return None

# Save cohort information
save_cohort_info('GSE161986', './preprocessed/Substance_Use_Disorder/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Substance_Use_Disorder', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Substance_Use_Disorder/trait_data/GSE161986.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM4929029': [1.0, 61.0, 1.0], 'GSM4929030': [0.0, 44.0, 1.0], 'GSM4929031': [0.0, 62.0, 1.0], 'GSM4929032': [1.0, 56.0, 1.0], 'GSM4929033': [0.0, 63.0, 1.0], 'GSM4929034': [1.0, 42.0, 1.0], 'GSM4929035': [0.0, 46.0, 1.0], 'GSM4929036': [0.0, 56.0, 1.0], 'GSM4929037': [1.0, 52.0, 1.0], 'GSM4929038': [0.0, 43.0, 1.0], 'GSM4929039': [1.0, 59.0, 1.0], 'GSM4929040': [1.0, 56.0, 1.0], 'GSM4929041': [1.0, 54.0, 1.0], 'GSM4929042': [1.0, 46.0, 1.0], 'GSM4929043': [1.0, 39.0, 1.0], 'GSM4929044': [1.0, 73.0, 1.0], 'GSM4929045': [0.0, 56.0, 1.0], 'GSM4929046': [0.0, 50.0, 1.0], 'GSM4929047': [1.0, 63.0, 1.0], 'GSM4929048': [1.0, 50.0, 1.0], 'GSM4929049': [1.0, 50.0, 1.0], 'GSM4929050': [1.0, 51.0, 1.0], 'GSM4929051': [1.0, 64.0, 1.0], 'GSM4929052': [1.0, 55.0, 1.0], 'GSM4929053': [0.0, 55.0, 1.0], 'GSM4929054': [0.0, 47.0, 1.0], 'GSM4929055': [0.0, 50.0, 1.0], 'GSM4929056': [0.0, 55.0, 1.0], 'GSM4929057': [1.0, 53.0, 1.0], 'GSM4929058': [0.0, 82.0, 1.0], 'GSM4929059': [0.0, 64.0, 1.0], 'GSM492

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at',
       '179_at', '1861_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the correct keys in the gene annotation dictionary
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to get gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Substance_Use_Disorder/gene_data/GSE161986.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data are severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Substance_Use_Disorder')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE161986', './preprocessed/Substance_Use_Disorder/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Substance_Use_Disorder/GSE161986.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Substance_Use_Disorder', the least common label is '0.0' with 17 occurrences. This represents 48.57% of the dataset.
The distribution of the feature 'Substance_Use_Disorder' in this dataset is fine.

Quartiles for 'Age':
  25%: 50.0
  50% (Median): 55.0
  75%: 61.5
Min: 39.0
Max: 82.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 35 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.

